# Clean_Data_GOOD

## Imported libraries

In [392]:
import pandas as pd #To work with dataframes
import numpy as np #To work with np.arrays

## Reading the data

In [393]:
df = pd.read_csv('./datasets/data_100000.csv')

## Preprocessing the data

In [394]:
pd.set_option('display.max_columns', None)

#### Dropping columns with excessive (or duplicate) information

In [395]:
"""
information about the dropped columns: 
- 'root_stone','root_grate','root_other','trnk_wire','trnk_light','trnk_other','brnch_ligh','brnch_shoe','brnch_othe': All of these
    columns are already described in the column 'problems'.
- 'state': All trees are in the same state (=New York) because it is an internal state research, This column would only be 
    usefull if the same research was applied in another state.

"""

df = df.drop(['root_stone',
              'root_grate',
              'root_other',
              'trnk_wire',
              'trnk_light',
              'trnk_other',
              'brnch_ligh',
              'brnch_shoe',
              'brnch_othe',
             'state'], axis=1)

#### Renaming columns for readability

In [396]:
df = df.rename(columns={'tree_dbh':'tree_dbh[inch]','stump_diam':'stump_diam[inch]','sidewalk':'sidewalk_damage'})

#### Replacing white spaces with np.nan value

In [397]:
df = df.replace(r'^\s*$', np.nan, regex=True)

#### Removing rows with non-valid values for the column tree_id

In [398]:
df.tree_id = df.tree_id.replace(r'^(\d){6,10}*$', np.nan, regex=True)
df = df.loc[df.tree_id != np.nan]

#### Setting the column tree_id as index

In [399]:
df = df.set_index("tree_id")

#### Creating an extra column genus_latin that contains the genus name of the species of the tree

In [400]:
df["genus_latin"] = df.spc_latin.str.split(" ").str.get(0)

#### Replacing values of the column sidewalk_damage with Yes/No depending on the value

In [401]:
df.sidewalk_damage = df.sidewalk_damage.replace(["Damage","NoDamage"], ["Yes","No"])

#### changing the column order for readability

In [402]:
cols = df.columns.tolist()
new_columns_order = [cols[0],cols[31],cols[8],cols[9],cols[6],cols[7],cols[14],cols[3],cols[4],
        cols[2],cols[1],cols[15],cols[16],cols[17],cols[18],cols[19],cols[20],cols[5],
            cols[10],cols[11],cols[12],cols[13],cols[24],cols[25],cols[26],cols[27],cols[28],
            cols[29],cols[30],cols[21],cols[22],cols[23]]

df = df.reindex(columns=new_columns_order)

#### Replacing the 0 values and values that have more than 4 digits in the columns tree_dbh and stump_diam to np.nan

In [403]:
df.tree_dbh = df["tree_dbh[inch]"].replace(0, np.nan)
df.stump_diam = df["stump_diam[inch]"].replace(0, np.nan)
df["tree_dbh[inch]"] = df["tree_dbh[inch]"].replace(r'^(\d){4,10}$', value = np.nan, regex=True)

C:\Users\VervoortLaptop\anaconda3\envs\Standard_Work_Env\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.
C:\Users\VervoortLaptop\anaconda3\envs\Standard_Work_Env\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


#### Checking the block_id,  zipcode, cb_num, borocode column for right amount of digits

In [404]:
df["block_id"] = df["block_id"].replace(r'^(\d){0,6}$', value = np.nan, regex=True)
df["block_id"] = df["block_id"].replace(r'^(\d){8,12}$',  value = np.nan, regex=True)

df["zipcode"] = df["zipcode"].replace(r'^(\d){0,4}$', value = np.nan, regex=True)
df["zipcode"] = df["zipcode"].replace(r'^(\d){6,10}$',  value = np.nan, regex=True)

df["cb_num"] = df["cb_num"].replace(r'^(\d){0,2}$', value = np.nan, regex=True)
df["cb_num"] = df["cb_num"].replace(r'^(\d){4,10}$',  value = np.nan, regex=True)

df["borocode"] = df["borocode"].replace(r'^(\d){3,10}$', value = np.nan, regex=True)

#### Replacing the values of steward with more logical values

In [405]:
df.steward = df.steward.replace(['None', '1or2', '3or4', '4orMore'], ["0","1 or 2","3 or 4","+4" ])

#### Checking the values of columns: status, health, guards, steward, user_type and curb_loc for valid values

In [406]:
df.loc[~df["status"].isin(["Alive","Dead","Stump"]), "status"] = np.nan
df.loc[~df["health"].isin(["Good","Fair","Poor"]), "health"] = np.nan
df.loc[~df["guards"].isin(['None', 'Helpful', 'Harmful', 'Unsure']), "guards"] = np.nan
df.loc[~df["steward"].isin(["0","1 or 2","3 or 4","+4" ]), "steward"] = np.nan
df.loc[~df["user_type"].isin(['TreesCount Staff', 'Volunteer', 'NYC Parks Staff']), "user_type"] = np.nan
df.loc[~df["curb_loc"].isin(['OnCurb', 'OffsetFromCurb']), "curb_loc"] = np.nan

#### Changing the dataframe to dtype: "string"

In [407]:
df = df.applymap(str)

#### Removing all the excessive white spaces

In [408]:
df = df.applymap(str.strip).rename(columns=str.strip)

#### Replacing all the np.nan values of the dataframe by "Information not available"

In [409]:
df = df.replace(np.nan, "Information not available")

#### Changing the dtypes of each column to the correct form and consolidate them if necessary

In [410]:
df.created_at = df.created_at.astype('datetime64')
df.genus_latin = df.genus_latin.astype('string').str.capitalize()
df.spc_latin = df.spc_latin.astype('string').str.capitalize()
df.spc_common = df.spc_common.astype('string').str.capitalize()
df.status = df.status.astype('string').str.capitalize()
df.health = df.health.astype('string').str.capitalize()
df.problems = df.problems.astype('string')
df["tree_dbh[inch]"] = df["tree_dbh[inch]"].astype('int8')
df["stump_diam[inch]"] = df["stump_diam[inch]"].astype('int8')
df.address = df.address.astype('string').str.capitalize()
df.zipcode = df.zipcode.astype('int16')
df.zip_city = df.zip_city.astype('string')
df.cb_num = df.cb_num.astype('int16')
df.boroname = df.boroname.astype('string').str.capitalize()
df.borocode = df.borocode.astype('int8')
df.curb_loc = df.curb_loc.astype('string').str.capitalize()
df.steward = df.steward.astype('string').str.lower()
df.guards = df.guards.astype('string').str.capitalize()
df.sidewalk_damage = df.sidewalk_damage.astype('string').str.capitalize()
df.user_type = df.user_type.astype('string').str.capitalize()
df.nta = df.nta.astype('string').str.upper()
df.nta_name = df.nta_name.astype('string').str.capitalize()
df.boro_ct = df.boro_ct.astype('int64')
df.latitude  = df.latitude .astype("float64") 
df.longitude = df.longitude.astype("float64") 
df.x_sp = df.x_sp.astype("float64") 
df.y_sp = df.y_sp.astype("float64")                                               
df.cncldist = df.cncldist.astype('int8')
df.st_assem  = df.st_assem .astype('int8')
df.st_senate  = df.st_senate .astype('int8')

#### Splitting the information within the problems column and making a list of them

In [411]:
df.problems = list(df.problems.str.split(","))

#### Creating a second dataset df_staff based on df with rows where user_type is not 'Volunteer'

In [412]:
df_staff = df[df.user_type != "Volunteer"]

## Saving the preprocessed data as .csv file

In [295]:
df.to_csv("Clean_data_GOOD_ENOUGH_not_official.csv")
df_staff.to_csv("Clean_data_GOOD_ENOUGH_official.csv")